In [2]:
from sklearn.neural_network import MLPClassifier
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split


In [3]:
import matplotlib.pyplot as plt
#%matplotlib inline 
import seaborn as sns

In [4]:
df_skroutz_word2vec = pd.read_parquet('../../Datos/embeddings/df_skroutz_word2vec.parquet', engine='pyarrow')

In [5]:
df_skroutz_word2vec.groupby("same_product").count()

,ProductID,ProductID2,ProductID_word2vec,ProductID2_word2vec
same_product,,,,
False,31762,31762,31762,31762
True,31762,31762,31762,31762


In [6]:
df_skroutz_word2vec["features"] = df_skroutz_word2vec["ProductID_word2vec"].apply(lambda row: row.tolist()) + df_skroutz_word2vec["ProductID2_word2vec"].apply(lambda row: row.tolist())
df_skroutz_word2vec["features"]=df_skroutz_word2vec["features"].apply(lambda row: np.array(row))

In [7]:
X = df_skroutz_word2vec["features"].explode().to_numpy().reshape(df_skroutz_word2vec.shape[0],600).astype("float")
y = df_skroutz_word2vec[["same_product"]].astype("int").to_numpy().reshape(df_skroutz_word2vec.shape[0],1)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42, stratify= y)

In [9]:
from sklearn.model_selection import GridSearchCV
clf = MLPClassifier(hidden_layer_sizes=(600,100,50,1))
param_grid = {"activation": ["identity","logistic","tanh","relu"], 
              "solver":["lbfgs", "sgd","adam"], "alpha":[0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1,0.000001], 
              "learning_rate":["constant", "invscaling", "adaptive"],
              "learning_rate_init":[0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1,0.000001],
              "power_t":[0.5, 0.1, 0.9,0.001,0.005],"max_iter":[5,10,20,50], "early_stopping": [True],"n_iter_no_change":[5]}
grid_search = GridSearchCV(clf, param_grid, verbose=2, n_jobs=-1, cv=3, scoring="balanced_accuracy")

In [ ]:
grid_search.fit(X_train, np.ravel(y_train))